<a href="https://colab.research.google.com/github/ITU-Business-Analytics-Team/Business_Analytics_for_Professionals/blob/main/Part%20II%20%3A%20Business%20Applications/Chapter%2011%3A%20Supply%20Chain%20Analytics/11_6_Aggregate_Production_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Supply Chain Analytics**
## Aggregate Production Planning

### **Case Study: A bicycle production company**

In [ ]:
import numpy as np
import pandas as pd
import math
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
# Import Demand Forecast Data

url="https://docs.google.com/spreadsheets/d/1JJ3iffPxOgLvVSCvhXBijmsXa2s7qONy/edit?usp=sharing&ouid=108589602591644119588&rtpof=true&sd=true"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
D = pd.read_excel(path, header=None) #product demands

num_of_periods=D.shape[1] #Currently there are 15 periods
num_of_products=D.shape[0] #Currently there are 3 type of product families

for i in range(num_of_periods):
  D.columns=['Period {}'.format(i+1) for i in range(num_of_periods)]
for i in range(num_of_products):
  D.index=['Product {}'.format(i+1) for i in range(num_of_products)]
  
D.head()

,Period 1,Period 2,Period 3,Period 4,Period 5,Period 6,Period 7,Period 8,Period 9,Period 10,Period 11,Period 12,Period 13,Period 14,Period 15
Product 1,1000,3300,5800,3200,2200,1000,1500,2400,3000,3500,4400,6000,5500,3000,2200
Product 2,2000,3600,4200,2300,3300,1000,1600,3500,3400,3500,2700,5000,5700,4500,5600
Product 3,1500,2500,1600,4200,2000,2000,1700,2000,3000,3700,1800,5000,5700,1000,1600


We have demand data of 3 different products for 15 periods as given above.



All parameters related to the problem and their definitions are given in the following table.

Parameter Name  | Parameter Description 
-------------------|------------------
$t$ | 1,2,...,T where T is the planning horizon
$i$ | 1,2,...,I where I is the number of product families
$D_{it}$ | Product demand for product $i$ in time $t$
$R_i$       |The number of units for product $i$ that one worker can produce in a period on regular time
$O_i$ | The number of units for product $i$ that one worker can produce in a period on overtime
$II_i$ | The initial inventory level for product $i$
$DI_i$ | The desired inventory level at the end of the planning horizon for product $i$
$CR_i$ | The unit production cost for product $i$ per period on regular time
$CO_i$ | The unit production cost for product $i$ per period on overtime
$CS_i$ | The unit production cost for product $i$ per period on subcontracting
$CK_i$ | The cost per period of carrying one unit of inventory for product $i$
$IW$ | The initial workforce level
$DW$ | The desired workforce level at the end of the planning horizon
$CH$ | The cost of hiring a worker 
$CF$ | The cost of firing a worker
$CW$ | The wage cost of a worker in a period





In [ ]:
# Import Parameters

url = "https://docs.google.com/spreadsheets/d/1AV4Cn4ocg-Zqqv3C1t-KiVnds19aMejq/edit?usp=sharing&ouid=116301364204212013828&rtpof=true&sd=true"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
parameters = pd.read_excel(path)
for i in range(num_of_products):
  parameters.index=['Product {}'.format(i+1) for i in range(num_of_products)]
parameters.head()

,R,O,II,DI,CR,CO,CS,CK
Product 1,10,8,1000,0,30,40,500,15
Product 2,8,6,1150,0,35,44,550,20
Product 3,9,7,1800,0,20,38,440,13


In [ ]:
IW=100
DW=140
CH=3500
CF=1000
CW=4000

Decision variables are given below.

Decision Variable Name  | Decision Variable Description 
-------------------|------------------
$H_{t}$       | The number of workers hired in period $t$
$F_{t}$       | The number of workers fired in period $t$
$W_{t}$       |The number of workers employed in period $t$
$r_{it}$ | The number of units produced for product $i$ on regular time in period $t$
$o_{it}$ | The number of units produced for product $i$ on overtime in period $t$
$s_{it}$ | The number of units produced for product $i$ on subcontracting in period $t$
$k_{it}$ | The number of units stored for product $i$ in inventory at the end of period $t$

Objective is minimizing the total cost of hiring, firing, wages, total regular time, extra time, subcontracting and transporation.

In [ ]:
#Linear Programming with Minimization Objective

!pip install pulp
import pulp as plp
from pulp import LpProblem, LpVariable, LpMinimize,LpStatus, lpSum

     |████████████████████████████████| 41.2 MB 76 kB/s 


In [ ]:
# Create the model
model = LpProblem(name="production-scheduling-problem", sense=LpMinimize)

# Initialize the decision variables
H = {t: plp.LpVariable(name=f"H_{t}", lowBound=0, cat=plp.LpInteger) for t in range(1, num_of_periods+1)} #range function does not involve the last term
F = {t: plp.LpVariable(name=f"F_{t}", lowBound=0, cat=plp.LpInteger) for t in range(1, num_of_periods+1)} 
W = {t: plp.LpVariable(name=f"W_{t}", lowBound=0, cat=plp.LpInteger) for t in range(num_of_periods+1)}
r = {(i,t): plp.LpVariable(name=f"r_{i}_{t}", lowBound=0, cat=plp.LpInteger) for i in range(1,num_of_products+1) for t in range(1, num_of_periods+1)}
o = {(i,t): plp.LpVariable(name=f"o_{i}_{t}", lowBound=0, cat=plp.LpInteger) for i in range(1,num_of_products+1) for t in range(1, num_of_periods+1)}
s = {(i,t): plp.LpVariable(name=f"s_{i}_{t}", lowBound=0, cat=plp.LpInteger) for i in range(1,num_of_products+1) for t in range(1, num_of_periods+1)}
k = {(i,t): plp.LpVariable(name=f"k_{i}_{t}", lowBound=0, cat=plp.LpInteger) for i in range(1,num_of_products+1) for t in range(num_of_periods+1)}



In [ ]:
#Inventory Constraint
for i in range(1,num_of_products+1):
  for t in range(1,num_of_periods+1):
    model += (k[i,t-1]+r[i,t]+o[i,t]+s[i,t]-D.iat[i-1,t-1]-k[i,t]==0 , "Inventory constraint for %d and %d" % (i,t))

#Regular Time and Overtime Production Constraints
for t in range(1,num_of_periods+1):
  model += (plp.lpSum(r[i,t]*float(parameters.R[i-1])**(-1) for i in range(1,num_of_products+1)) <= W[t] , "Regular time production constraints for %d" %t )
  model += (plp.lpSum(o[i,t]*float(parameters.O[i-1])**(-1) for i in range(1,num_of_products+1)) <= W[t] , "Overtime production constraints for %d" %t )   

#Subcontracting Constraints
for t in range(1,num_of_periods+1):
  model += (plp.lpSum(s[i,t] for i in range(1,num_of_products+1)) <= 1000 , "Subcontructing constraints for %d" %t )

#Workforce level change constraints
for t in range(1,num_of_periods+1):
  model += (W[t-1]+H[t]-F[t]==W[t], "Workforce level change constraint for %d" %t)

#Initializing constraints
model += (W[0]==IW, "Initializing constraint of workforce level for the beginning of the time horizon")
model += (W[num_of_periods]==DW, "Initializing constraint of workforce level for the end of time horizon")

for i in range(1,num_of_products+1):
  model += (k[i,0] == parameters.II[i-1], "Initial inventory for product family %d" %i) 
  model += (k[i,num_of_periods] == parameters.DI[i-1], "Ending inventory for product family %d" %i)

#Set objective
model += plp.lpSum(CH*H[t]+CF*F[t]+CW*W[t]+parameters.CR[i-1]*r[i,t]+parameters.CO[i-1]*o[i,t]+parameters.CS[i-1]*s[i,t]+ parameters.CK[i-1]*k[i,t] for i in range(1,num_of_products+1) for t in range(1, num_of_periods+1) )

In [ ]:
#Solve model
model.solve()
# Get the results
print(f"Status: {model.status}, {LpStatus[model.status]}")
print(f"Objective: {model.objective.value()}")

Status: 1, Optimal
Objective: 109210929.0


In [ ]:
#The values of decision variables
for v in model.variables():
  print(v.name, "=", v.varValue)

F_1 = 0.0
F_10 = 0.0
F_11 = 0.0
F_12 = 0.0
F_13 = 0.0
F_14 = 0.0
F_15 = 420.0
F_2 = 0.0
F_3 = 0.0
F_4 = 0.0
F_5 = 0.0
F_6 = 0.0
F_7 = 0.0
F_8 = 0.0
F_9 = 0.0
H_1 = 0.0
H_10 = 0.0
H_11 = 0.0
H_12 = 0.0
H_13 = 0.0
H_14 = 0.0
H_15 = 0.0
H_2 = 387.0
H_3 = 73.0
H_4 = 0.0
H_5 = 0.0
H_6 = 0.0
H_7 = 0.0
H_8 = 0.0
H_9 = 0.0
W_0 = 100.0
W_1 = 100.0
W_10 = 560.0
W_11 = 560.0
W_12 = 560.0
W_13 = 560.0
W_14 = 560.0
W_15 = 140.0
W_2 = 487.0
W_3 = 560.0
W_4 = 560.0
W_5 = 560.0
W_6 = 560.0
W_7 = 560.0
W_8 = 560.0
W_9 = 560.0
k_1_0 = 1000.0
k_1_1 = 724.0
k_1_10 = 2460.0
k_1_11 = 2540.0
k_1_12 = 1020.0
k_1_13 = 0.0
k_1_14 = 1080.0
k_1_15 = 0.0
k_1_2 = 1320.0
k_1_3 = 0.0
k_1_4 = 0.0
k_1_5 = 0.0
k_1_6 = 0.0
k_1_7 = 4.0
k_1_8 = 0.0
k_1_9 = 1480.0
k_2_0 = 1150.0
k_2_1 = 0.0
k_2_10 = 4.0
k_2_11 = 2240.0
k_2_12 = 2720.0
k_2_13 = 2500.0
k_2_14 = 3480.0
k_2_15 = 0.0
k_2_2 = 0.0
k_2_3 = 0.0
k_2_4 = 0.0
k_2_5 = 1.0
k_2_6 = 1.0
k_2_7 = 1.0
k_2_8 = 0.0
k_2_9 = 0.0
k_3_0 = 1800.0
k_3_1 = 1316.0
k_3_10 = 14138.0
k_3_

In [ ]:
#To check the constraints
for c in model.constraints:
  print(model.constraints[c])

k_1_0 - k_1_1 + o_1_1 + r_1_1 + s_1_1 = 1000
k_1_1 - k_1_2 + o_1_2 + r_1_2 + s_1_2 = 3300
k_1_2 - k_1_3 + o_1_3 + r_1_3 + s_1_3 = 5800
k_1_3 - k_1_4 + o_1_4 + r_1_4 + s_1_4 = 3200
k_1_4 - k_1_5 + o_1_5 + r_1_5 + s_1_5 = 2200
k_1_5 - k_1_6 + o_1_6 + r_1_6 + s_1_6 = 1000
k_1_6 - k_1_7 + o_1_7 + r_1_7 + s_1_7 = 1500
k_1_7 - k_1_8 + o_1_8 + r_1_8 + s_1_8 = 2400
k_1_8 - k_1_9 + o_1_9 + r_1_9 + s_1_9 = 3000
-k_1_10 + k_1_9 + o_1_10 + r_1_10 + s_1_10 = 3500
k_1_10 - k_1_11 + o_1_11 + r_1_11 + s_1_11 = 4400
k_1_11 - k_1_12 + o_1_12 + r_1_12 + s_1_12 = 6000
k_1_12 - k_1_13 + o_1_13 + r_1_13 + s_1_13 = 5500
k_1_13 - k_1_14 + o_1_14 + r_1_14 + s_1_14 = 3000
k_1_14 - k_1_15 + o_1_15 + r_1_15 + s_1_15 = 2200
k_2_0 - k_2_1 + o_2_1 + r_2_1 + s_2_1 = 2000
k_2_1 - k_2_2 + o_2_2 + r_2_2 + s_2_2 = 3600
k_2_2 - k_2_3 + o_2_3 + r_2_3 + s_2_3 = 4200
k_2_3 - k_2_4 + o_2_4 + r_2_4 + s_2_4 = 2300
k_2_4 - k_2_5 + o_2_5 + r_2_5 + s_2_5 = 3300
k_2_5 - k_2_6 + o_2_6 + r_2_6 + s_2_6 = 1000
k_2_6 - k_2_7 + o_2_7 + r

### **Level Sales and Operations Planning Heuristic**

This heuristic is applied for the first product family.


In [ ]:
import datetime
from dateutil import relativedelta

#Number of work days for 15 months starting from 2021 January

work_days_per_period=[]
start_date = datetime.datetime(2021,1,1)

for t in range(num_of_periods):
  end_date= start_date + relativedelta.relativedelta(months=1)
  work_days_per_period.append(np.busday_count('%d-0%d' %(start_date.year, start_date.month) if start_date.month<10 else '%d-%d' %(start_date.year, start_date.month),'%d-0%d' %(end_date.year, end_date.month) if end_date.month<10 else '%d-%d' %(end_date.year, end_date.month)))
  start_date=end_date

total_work_days= np.busday_count('2021-01', '2022-04') #Total number of work days for 15 months starting from 2021 January

total_forecasted_demand=D.sum(axis=1)[0] #Total forecasted demand for Product Family 1

In [ ]:
daily_production_rate= (total_forecasted_demand - parameters.II[0])/total_work_days

production_per_period=[]
for t in range(num_of_periods):
  production_per_period.append(daily_production_rate*work_days_per_period[t])

reg_time_prod_rate=parameters.R[0] #Regular time production rate for Product Family 1
over_time_prod_rate=parameters.O[0] #Overtime production rate for Product Family 1
reg_time_prod_cost=parameters.CR[0] #Regular time production rate for Product Family 1
over_time_prod_cost=parameters.CO[0] #Overtime production rate for Product Family 1
holding_cost=parameters.CK[0]       #Holding cost for Product Family 1
backlog_cost=18

In [ ]:
#If overtime is allowed

workers_initial=100 
workers= math.ceil(daily_production_rate/(reg_time_prod_rate+ over_time_prod_rate)) 
hired=max(workers-workers_initial,0)
fired=max(workers_initial-workers,0)

RTP=[]
OTP=[]
for t in range(num_of_periods):
  RTP.append(workers*reg_time_prod_rate*work_days_per_period[t]) #Regular time production in period t.
  OTP.append(workers*over_time_prod_rate*work_days_per_period[t]) #Overtime production in period t.

inventory=[]
backlog=[]
initial_inventory=parameters.II[0]

for t in range(num_of_periods):
  if t == 0:
    temp_inv=initial_inventory+RTP[t]+OTP[t]-D.iat[0,t] 
    if temp_inv<0:
      inventory.append(0)
      backlog.append(abs(temp_inv))

    else:
      inventory.append(temp_inv)
      backlog.append(0)

  else:
    temp_inv=inventory[t-1]+RTP[t]+OTP[t]-D.iat[0,t]
    if temp_inv<0:
      inventory.append(0)
      backlog.append(abs(temp_inv))

    else:
      inventory.append(temp_inv)
      backlog.append(0)

cost_per_period_wo=[]
for t in range(num_of_periods):
  if t == 0:
    cost_per_period_wo.append(inventory[t]*holding_cost + workers*CW + backlog[t]*backlog_cost + RTP[t]*reg_time_prod_cost + OTP[t]*over_time_prod_cost + hired*CH + fired*CF)
  else:
    cost_per_period_wo.append(inventory[t]*holding_cost + workers*CW + backlog[t]*backlog_cost + RTP[t]*reg_time_prod_cost + OTP[t]*over_time_prod_cost)

total_cost_wo_ls = sum(cost_per_period_wo)



In [ ]:
#If overtime is NOT allowed 

workers= math.ceil(daily_production_rate/reg_time_prod_rate)
hired=max(workers-workers_initial,0)
fired=max(workers_initial-workers,0)
wage=4000

RTP=[]
for t in range(num_of_periods):
  RTP.append(workers*reg_time_prod_rate*work_days_per_period[t]) #Regular time production in period t.

inventory=[]
backlog=[]
initial_inventory=parameters.II[0]

for t in range(num_of_periods):
  if t == 0:
    temp_inv=initial_inventory+RTP[t]-D.iat[0,t]
    if temp_inv<0:
      inventory.append(0)
      backlog.append(abs(temp_inv))

    else:
      inventory.append(temp_inv)
      backlog.append(0)

  else:
    temp_inv=inventory[t-1]+RTP[t]-D.iat[0,t]
    if temp_inv<0:
      inventory.append(0)
      backlog.append(abs(temp_inv))

    else:
      inventory.append(temp_inv)
      backlog.append(0)

cost_per_period_no=[]
for t in range(num_of_periods):
  if t == 0:
    cost_per_period_no.append(inventory[t]*holding_cost + workers*CW + backlog[t]*backlog_cost + RTP[t]*reg_time_prod_cost + hired*CH + fired*CF)
  else:
    cost_per_period_no.append(inventory[t]*holding_cost + workers*CW + backlog[t]*backlog_cost + RTP[t]*reg_time_prod_cost)

total_cost_no_ls = sum(cost_per_period_no)

In [ ]:
#Decision

print("The total cost when overtime is allowed: %f" %total_cost_wo_ls)
print("The total cost when overtime is not allowed: %f" %total_cost_no_ls)

if total_cost_wo_ls<total_cost_no_ls:
  print("Best decision is allowing overtime.")
else:
  print("Best decision is not allowing overtime.")

The total cost when overtime is allowed: 3646630.000000
The total cost when overtime is not allowed: 3183250.000000
Best decision is not allowing overtime.


### **Chase Sales and Operations Planning Heuristic**

This heuristic is applied for the first product family.

In [ ]:
production=[]
safety_stock=[]
forecast=[]
ss=0.01 #safety stock percentage

for t in range(num_of_periods):
  forecast.append(D.iat[0,t]) #only the first row of product_demands array is used as the forecasts of first product family.
  safety_stock.append(forecast[t]*ss)
  production.append(forecast[t]+safety_stock[t])

In [ ]:
#If overtime is allowed
inventory=[]
workers=[]
hired=[]
fired=[]
RTP=[]
OTP=[]
cost_per_period_wo=[]

for t in range(num_of_periods):
  inventory.append(safety_stock[t])
  workers.append(math.ceil(production[t]/(work_days_per_period[t]*(reg_time_prod_rate+over_time_prod_rate))))
  if (t>0):
    hired.append(max(workers[t]-workers[t-1],0))
    fired.append(max(workers[t-1]-workers[t],0))
  else:
    hired.append(workers[t])
    fired.append(0)

  RTP.append(reg_time_prod_rate*workers[t]*work_days_per_period[t])
  OTP.append(over_time_prod_rate*workers[t]*work_days_per_period[t])
  cost_per_period_wo.append(holding_cost*inventory[t]+workers[t]*CW+hired[t]*CH+fired[t]*CF+RTP[t]*reg_time_prod_cost+OTP[t]*over_time_prod_cost)

total_cost_wo_cs=sum(cost_per_period_wo)

In [ ]:
#If overtime is not allowed

inventory=[]
workers=[]
hired=[]
fired=[]
RTP=[]
cost_per_period_no=[]

for t in range(num_of_periods):
  inventory.append(safety_stock[t])
  workers.append(math.ceil(production[t]/(work_days_per_period[t]*(reg_time_prod_rate))))
  if (t>0):
    hired.append(max(workers[t]-workers[t-1],0))
    fired.append(max(workers[t-1]-workers[t],0))
  else:
    hired.append(workers[t])
    fired.append(0)

  RTP.append(reg_time_prod_rate*workers[t]*work_days_per_period[t]) #NOT!! WORKERS INTEGER OLDUĞU İÇİN PRODUCTION İLE RTP EŞİT OLMAYABİLİR
  cost_per_period_no.append(holding_cost*inventory[t]+workers[t]*CW+hired[t]*CH+fired[t]*CF+RTP[t]*reg_time_prod_cost+OTP[t]*over_time_prod_cost)

total_cost_no_cs=sum(cost_per_period_no)


### **Results**

In [ ]:
#Decision

print("The total cost when overtime is allowed: %f" %total_cost_wo_cs)
print("The total cost when overtime is not allowed: %f" %total_cost_no_cs)

if total_cost_wo_cs<total_cost_no_cs:
  print("Best decision is allowing overtime.")
else:
  print("Best decision is not allowing overtime.")

The total cost when overtime is allowed: 2428860.000000
The total cost when overtime is not allowed: 3550060.000000
Best decision is allowing overtime.
